In [1]:
from enum import auto
import pandas as pd
import numpy as np
import urllib
import sqlalchemy as sa
import warnings
from unidecode import unidecode

warnings.filterwarnings("ignore")

In [2]:
def remove_accents(x):

    #convert plain text to utf-8
    u = unidecode(x, "utf-8")
    #convert utf-8 to normal text
    return (unidecode(u))

In [3]:
def HC_cleaning():
    HC=pd.read_csv(r"C:\Users\bustossanchez.9\OneDrive - Teleperformance\Procesos\Amynta\Scores\HC_The_Amynta_Group Deliverable.csv",  delimiter=';', encoding= 'unicode_escape')
    HC=HC.iloc[:,[0,2,6,8,10,14,18,26,28]]
    HC.columns=HC.columns.str.lower()
    
    HC.rename(columns={"agent_name":"agentname",
                        "vlookup":"agentid",
                        "tenure inactive (since ops)":"tenure"
                                              }, inplace=True)
        
    HC=HC.replace(",","0", regex=True)
    HC=HC.replace("-","0", regex=True)
    HC=HC.apply(lambda x: x.astype(str).str.lower())

    HC["agentname"]=HC["agentname"].apply(lambda x: remove_accents(x))
    HC['agentid']=HC["agentid"].astype('int64')
    return HC

In [82]:
HC=HC_cleaning()
RawData=pd.read_excel(f"SearchReport_2023_02_15_16_39.xlsx", header=4)

#Column names to lowercase, drop unecessary fields and replace name
RawData.columns=RawData.columns.str.lower()
RawData=RawData.drop(RawData.columns[[1,2,6]], axis=1)
RawData=RawData.drop(['contact link'], axis=1)
RawData.rename(columns={"login_id":"agentid",
                            },
                            inplace=True
    )

RawData.columns=[name.replace("categories.","").replace("`","") for name in RawData.columns]


#merge and create new table
RawUpdated=pd.merge(HC, RawData, on="agentid", how="right")

#Replacing hit and miss with 1's and 0's and leaving the columns as factors
RawUpdated.iloc[:,15:]=RawUpdated.iloc[:,15:].replace("Hit",1, regex=True).replace("Miss",0, regex=True)

#Rename columns by component name
RawUpdated.columns=RawUpdated.columns[:15].to_list()+[name.split(".")[2] if len(name.split("."))==3 else name.replace("."," ") for name in RawUpdated.columns[15:]]



#Drop duplicates by EurekaID and columns duplicated
RawUpdated.drop_duplicates(subset=["eureka id"], inplace=True)


#RawUpdated=RawUpdated.loc[:,~RawUpdated.columns.duplicated()]              We don't want to do this 


#Filtering by conditions

RawUpdated=RawUpdated[(RawUpdated["outliers excessive silence"]==0) 
                    &(RawUpdated["outliers longest contacts"]==0)
                    &(RawUpdated["agentid"]!=0)
                    &(RawUpdated["outliers spanish calls"]==0)]

#Sort and reset_index
RawUpdated=RawUpdated.sort_values(by="eureka id").reset_index(drop=True)

#Categories Lists

categories = RawUpdated.columns.tolist()[15:]
############################################################## WE DELETED THE [31], IT NEEDS TO BE FIXED 
categories_outliers= categories[-36:-23]
categories_agent_cps=categories[0:9]+categories[17:47]+categories[53:65]+categories[68:73]+categories[77:83]+categories[87:92]
categories_brand_cps=categories[97:108]
categories_channel_cps=categories[113:127]
categories_tpd_cps = categories[155:]

#Basic call info df

basic_info=['agentid','eureka id','percent silence','recordingdatetime']

# CPS

# 1.1 Agent_CPS
 
categories_agent_cps= basic_info + categories_agent_cps
#DataFrame
agent_cps=RawUpdated[categories_agent_cps]

# 1.1.1 Opening

opening_cps= ['agent demonstrates willingness to assist',
            'agent does not acknowledge the customer request',  ##negative??? 
            'agent does not conveys urgency to assist',  ## Negative??
            'agent uses effective listening skills',
            'the agent dont uses effective listening skills',  ## Negative??
            'unnecessary information',
            'thank you for calling']

agent_cps["opening_score_cps"]=0

for cat in opening_cps:
    agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (10/7) if x==1 else (0)) 
    agent_cps["opening_score_cps"]=agent_cps["opening_score_cps"]+agent_cps[cat+'_score']
    

# 1.1.2 Investigation

    #1.1.2.1 Personalized Interaction 
agent_cps["personalized_interaction_score_cps"]=0  

agent_cps['agent asks the required information_score'] = agent_cps['agent asks the required information'].apply(lambda x: 4 if x==1 else 0)
agent_cps["personalized_interaction_score_cps"]=agent_cps["personalized_interaction_score_cps"]+agent_cps['agent asks the required information_score']

agent_cps['callers name_score'] = agent_cps['callers name'].apply(lambda x: 2 if x==1 else 0) 
agent_cps["personalized_interaction_score_cps"]=agent_cps["personalized_interaction_score_cps"]+agent_cps['callers name_score']

    #1.1.2.2 File a Claim
agent_cps["file_a_claim_score_cps"]=0

file_a_claim = ['adh question',
                'commerical vs personal',
                'how was the unit being used',
                'model number',
                'the date when the problem occurred',
                'what led up to issue']

for cat in file_a_claim:
    agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 2 if x==1 else (0)) 
    agent_cps["file_a_claim_score_cps"]=agent_cps["file_a_claim_score_cps"]+agent_cps[cat+'_score']

    #1.1.2.3 Damage from SC   
agent_cps['damage_from_sc_score_cps']=agent_cps['have you contacted the sc about the damage'].apply(lambda x: 4 if x==1 else (0))
    
    #1.1.2.4 Offer Status 
agent_cps["offer_stat_score_cps"]=0  

offer_stat=['check has been approved',
            'gift card has been approved',
            'replacement',
            'store credit has been approved']

for cat in offer_stat:
    agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 0.75 if x==1 else (0)) 
    agent_cps["offer_stat_score_cps"]=agent_cps["offer_stat_score_cps"]+agent_cps[cat+'_score']

    #1.1.2.5 Registration
agent_cps["registration_score_cps"]=0

registration=[  'how much pay for the warranty',
                'product registration',
                'what is the model number',
                'what is the name of the product',
                'what is the purchase date']

for cat in registration:
    agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 0.8 if x==1 else (0)) 
    agent_cps["registration_score_cps"]=agent_cps["registration_score_cps"]+agent_cps[cat+'_score']

    #1.1.2.6 Near Store

agent_cps['near_store_score_cps']=agent_cps['near store'].apply(lambda x: 2 if x==1 else (0))   

    #1.1.2.7 Food Loss Claim 
agent_cps["food_loss_claim_investigation_score_cps"]=0

food_loss_claim_investigation=[ 'itemized list of food loss',
                                'the agent tells the customer to send the info',
                                'the failure is a no cooling or no power']

for cat in food_loss_claim_investigation:
    agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 0.8 if x==1 else (0)) 
    agent_cps["food_loss_claim_investigation_score_cps"]=agent_cps["food_loss_claim_investigation_score_cps"]+agent_cps[cat+'_score']                        

    #1.1.2.8 FINAL SCORE INVESTIGATION
agent_cps["investigation_score_cps"]=(  agent_cps["food_loss_claim_investigation_score_cps"]+
                                        agent_cps['near_store_score_cps']+
                                        agent_cps["registration_score_cps"]+
                                        agent_cps["personalized_interaction_score_cps"] +
                                        agent_cps["offer_stat_score_cps"] +
                                        agent_cps['damage_from_sc_score_cps']+
                                        agent_cps["file_a_claim_score_cps"])




#1.1.3 Resolution

#1.1.3.1 Dead air time  
agent_cps['dead_air_time_score_cps']=agent_cps['proper hold techniques'].apply(lambda x: 2 if x==1 else (0)) 

#1.1.3.2 Response time 
agent_cps['sc_response_time_score_cps']=agent_cps['response time'].apply(lambda x: 3 if x==1 else (0)) 

#1.1.3.3 SC contact info 
agent_cps['sc_contact_info_score_cps']=agent_cps['contact info'].apply(lambda x: 4 if x==1 else (0)) 

#1.1.3.4 Appointment info
agent_cps['sc_appointment_info_score_cps']=agent_cps['sc appointment template'].apply(lambda x: 6 if x==1 else (0)) 

#1.1.3.5 Damage from SC 
'3 resolution cps damage from service center'
agent_cps['damage_from_sc_score_cps']=agent_cps['damage'].apply(lambda x: 3 if x==1 else (0)) 

#1.1.3.5 Food Lost Resolution
agent_cps['food_loss_resolution_score_cps']=agent_cps['recite script'].apply(lambda x: 3 if x==1 else (0)) 
 
#1.1.3.6 FINAL SCORE RESOLUTION

agent_cps["resolution_score_cps"] = (   agent_cps['food_loss_resolution_score_cps']+
                                        agent_cps['damage_from_sc_score_cps']+
                                        agent_cps['sc_appointment_info_score_cps']+
                                        agent_cps['sc_contact_info_score_cps']+
                                        agent_cps['sc_response_time_score_cps']+
                                        agent_cps['dead_air_time_score_cps'])
# 1.1.4 Closing   

closing_cps = ['proper closing',
                'proper transfer' ]

agent_cps["closing_score_cps"]=0

for cat in closing_cps:
    agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 2 if x==1 else (0)) 
    agent_cps["closing_score_cps"]=agent_cps["closing_score_cps"]+agent_cps[cat+'_score']


# 1.1.4 Legal 

legal_cps =[    'attorney',
                'confidential information',
                'derogatory manner',
                'internal procedure',
                'social network']

agent_cps["legal_score_cps"]=0

for cat in legal_cps:
    agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 5 if x==1 else (0)) 
    agent_cps["legal_score_cps"]=agent_cps["legal_score_cps"]+agent_cps[cat+'_score']

#1.1.5 Amynta Experiences

amynta_experiences_cps=['blames the customer',
                        'colloquial words',
                        'communicates with respect',
                        'empathy and interest']

agent_cps["experiences_score_cps"]=0

for cat in amynta_experiences_cps:
    agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 2 if x==1 else (0)) 
    agent_cps["experiences_score_cps"]=agent_cps["experiences_score_cps"]+agent_cps[cat+'_score']

#1.1.6 FINAL SCORE AGENT

agent_cps["agent_score_cps"] = (agent_cps["experiences_score_cps"]+
                                agent_cps["legal_score_cps"]+
                                agent_cps["closing_score_cps"]+
                                agent_cps["resolution_score_cps"]+
                                agent_cps["investigation_score_cps"]+
                                agent_cps["opening_score_cps"])


#1.2 Branding 

categories_brand_cps = basic_info + categories_brand_cps
branding_cps=RawUpdated[categories_brand_cps]

branding_cps["branding_score_cps"]=0

for cat in categories_brand_cps:
    if cat in [ 'the customer cant reach the sc',
                'the customer perceives unsatisfactory service',
                'there is no timely response from sc']:
        branding_cps[cat+'_score']=branding_cps[cat].apply(lambda x: 8 if x==1 else (0))
        branding_cps["branding_score_cps"]=branding_cps["branding_score_cps"]+branding_cps[cat+'_score']

    if cat in [ 'contract expired',
                'contract fulfilled',
                'contract incomplete',
                'did not review terms and conditions',
                'no contract found',
                'no cover issue']:
        branding_cps[cat+'_score']=branding_cps[cat].apply(lambda x: (76/6) if x==1 else (0))
        branding_cps["branding_score_cps"]=branding_cps["branding_score_cps"]+branding_cps[cat+'_score']

#1.3 Channel 

categories_channel_cps = basic_info + categories_channel_cps
channel_cps=RawUpdated[categories_channel_cps]

channel_cps["channel_score_cps"]=0 

for cat in categories_channel_cps:
    if cat in [ 'been on hold for so long time on the queue',
                'been on hold or waiting',
                'i dont want to wait',
                'long waiting time or callback',
                'request never answered',
                'there is no answer',
                'tired of waiting',
                'website not working']:

        channel_cps[cat+'_score']=channel_cps[cat].apply(lambda x: 8.5 if x==1 else (0))
        channel_cps["channel_score_cps"]=channel_cps["channel_score_cps"]+channel_cps[cat+'_score']
        
    if cat in[  'application is not running',
                'delay when transfering the call',
                'not good communication',
                'the customer express any disagreement']:

        channel_cps[cat+'_score']=channel_cps[cat].apply(lambda x: 8 if x==1 else (0))
        channel_cps["channel_score_cps"]=channel_cps["channel_score_cps"]+channel_cps[cat+'_score']


channel_cps.to_excel('channel_cps.xlsx')
branding_cps.to_excel('branding_cps.xlsx')
agent_cps.to_excel('agent_cps.xlsx')

# GMAN

# 1.1 Agent_CPS

In [87]:
for cat in categories_channel_cps:
    print (cat)
    if cat in [ 'been on hold for so long time on the queue',
                'been on hold or waiting',
                'i dont want to wait',
                'long waiting time or callback',
                'request never answered',
                'there is no answer',
                'tired of waiting',
                'website not working']:
        print(cat)
        channel_cps[cat+'_score']=channel_cps[cat].apply(lambda x: 8.5 if x==1 else (0))
        channel_cps["channel_score_cps"]=channel_cps["channel_score_cps"]+channel_cps[cat+'_score']
        print(cat)
        
    if cat in[  'application is not running',
                'delay when transfering the call',
                'not good communication',
                'the customer express any disagreement']:

        channel_cps[cat+'_score']=channel_cps[cat].apply(lambda x: 8 if x==1 else (0))
        channel_cps["channel_score_cps"]=channel_cps["channel_score_cps"]+channel_cps[cat+'_score']

agentid
eureka id
percent silence
recordingdatetime
8 channel transversal cps complaining during the interaction
been on hold for so long time on the queue
been on hold for so long time on the queue
been on hold for so long time on the queue
been on hold or waiting
been on hold or waiting
been on hold or waiting
i dont want to wait
i dont want to wait
i dont want to wait
long waiting time or callback
long waiting time or callback
long waiting time or callback
request never answered
request never answered
request never answered
there is no answer
there is no answer
there is no answer
tired of waiting
tired of waiting
tired of waiting
website not working
website not working
website not working
8 channel transversal cps hard communication
application is not running
delay when transfering the call
not good communication
the customer express any disagreement


In [116]:
#categories_agent_cps[13:46]
'personalized_interaction_score_cps'
'file_a_claim_score_cps'
'damage_from_sc_score_cps'
'offer_stat_score_cps'
'registration_score_cps'
'food_loss_claim_investigation_score_cps'

In [135]:
for name in categories_agent_cps[13:43]:
    print(agent_cps[name].nunique())

2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
1
2
1
2
2
2
2
2
2
2
2
2


In [76]:
categories_agent_cps[50:]

['sc appointment template',
 '3 resolution cps sc contact info',
 'contact info',
 '3 resolution cps sc response time',
 'response time',
 '4 closing cps closing call',
 'proper closing',
 'proper transfer',
 '4 closing cps transfering cat cps',
 'transfer',
 '5 legal cps legal responsibility',
 'attorney',
 'confidential information',
 'derogatory manner',
 'internal procedure',
 'social network',
 '6 amynta experiences cps amynta experiences',
 'blames the customer',
 'colloquial words',
 'communicates with respect',
 'empathy and interest']

In [61]:
categories_brand_cps

['agentid',
 'eureka id',
 'percent silence',
 'recordingdatetime',
 '7 branding transversal cps contract terms and conditions',
 'contract expired',
 'contract fulfilled',
 'contract incomplete',
 'did not review terms and conditions',
 'no contract found',
 'no cover issue',
 '7 branding transversal cps service center issues',
 'the customer cant reach the sc',
 'the customer perceives unsatisfactory service',
 'there is no timely response from sc']

In [23]:
categories = RawUpdated.columns.tolist()[15:]

#75
#139-140



43
43


In [39]:
categories[155:]

['tpc tpc acccount updates',
 'tpc account updates',
 'tpc tpc cancelation request',
 'cancelation',
 'tpc tpc claim status  offer status',
 'tpc tpc claim status  offer status claim status or offer status ',
 'tpc tpc claim status appointment status',
 'agent response',
 'appointment status',
 'tpc tpc claim status parts status',
 'parts status',
 'tpc tpc claim status service center',
 'status service center',
 'tpc tpc file a claim',
 'file a claim tpc',
 'tpc tpc general inquiry',
 'general inquiry',
 'tpc tpc registration',
 'tpc registration']

In [6]:
def main(date):
    date=date.replace("/","-")
    HC=HC_cleaning()
    
    #Edit when the download.py is built    
    #RawData=pd.read_excel(f"Download folder\\Downloads Callminer_v2\\{date}\\SearchReport{date}.xlsx", header=4)
    RawData=pd.read_excel(f"SearchReport_2023_02_14_16_52.xlsx", header=4)

    RawData=RawData.drop(RawData.columns[[1,2,6]], axis=1)

    RawData.rename(columns={,
                            RawData.columns[1]:"email"},
                            inplace=True
    )

    #Column names to lowercase and replace names
    RawData.columns=RawData.columns.str.lower()

    RawData.columns=[name.replace("categories.","").replace("`","") for name in RawData.columns]

    #merge and create new table
    RawUpdated=pd.merge(HC_rel, RawData, on="C", how="right")


    #Replacing hit and miss with 1's and 0's and leaving the columns as factors
    RawUpdated.iloc[:,24:]=RawUpdated.iloc[:,24:].replace("Hit",1, regex=True).replace("Miss",0, regex=True)

    #Rename columns by component name
    RawUpdated.columns=RawUpdated.columns[:24].to_list()+[name.split(".")[2] if len(name.split("."))==3 else name.replace("."," ") for name in RawUpdated.columns[24:]]

    #Drop duplicates by EurekaID and columns duplicated
    RawUpdated.drop_duplicates(subset=["eureka id"], inplace=True)
    RawUpdated=RawUpdated.loc[:,~RawUpdated.columns.duplicated()]

    #Filtering by conditions
    #RawUpdated=RawUpdated[(RawUpdated["higher percentage silence"]!=0) 
    #                    &(RawUpdated["average confidence"]!=0)
    #                    &(RawUpdated["word count"]!=0)]

    def swap_columns(df, col1, col2):
        col_list = list(df.columns)
        x, y = col_list.index(col1), col_list.index(col2)
        col_list[y], col_list[x] = col_list[x], col_list[y]
        df = df[col_list]
        return df

    #Swap email and agentname columns
    RawUpdated=swap_columns(RawUpdated, "email","agentname")

    #Sort and reset_index
    RawUpdated=RawUpdated.sort_values(by="eureka id").reset_index(drop=True)

    auto_fail=RawUpdated[["datetime", "eureka id","delivery misbehavior", "health", "legal", "blame customer", "describes doordash in a derogatory manner", 
            "profanity from agent to customer", "rude/arrogant", "agent is yelling and screaming", "confidential information", "covid 19"]]

    auto_fail.columns=[name.replace(" ","_").replace("/","").replace("^_","").replace("_$","") for name in auto_fail.columns]

    auto_fail=auto_fail.melt(id_vars=["datetime","eureka_id"])

    auto_fail=auto_fail[auto_fail["value"]>0]
    auto_fail["variable"]=auto_fail["variable"].str.replace("_", " ").str.replace("&"," ").str.capitalize()


    auto_fail=auto_fail.sort_values(by=["datetime","eureka_id","variable"]).reset_index(drop=True)

    #Security Riders/Informative 
    sec_riders=RawUpdated[["datetime","eureka id","call avoidance","call unexpetedly disconnected",
            "higher percentage silence", "sil hold", "sil transfer","lower avg confidence"]]

    sec_riders.columns=[name.replace(" ","_").replace("/","").replace("^_","").replace("_$","") for name in sec_riders.columns]

    sec_riders=sec_riders.melt(id_vars=["datetime","eureka_id"])

    sec_riders=sec_riders[sec_riders["value"]>0]
    sec_riders["variable"]=sec_riders["variable"].str.replace("_", " ").str.replace("&"," ").str.capitalize()


    sec_riders=sec_riders.sort_values(by=["datetime","eureka_id","variable"]).reset_index(drop=True)

SyntaxError: invalid syntax (<ipython-input-6-6fca9852cb23>, line 11)